## 영화 리뷰 긍정부정 분류 RNN 모델 - 데이터 전 처리

영화 리뷰 데이터를 바탕으로 감정 분석을 하는 모델을 학습 시켜 보겠습니다.   
영화 리뷰와 같은 자연어 자료는 곧 단어의 연속적인 배열로써, 시계열 자료라고 볼 수 있습니다.   
즉, 시계열 자료(연속된 단어)를 이용해 리뷰에 내포된 감정(긍정, 부정)을 예측하는 분류기를 만들어 보겠습니다.    

데이터셋은 IMDB 영화 리뷰 데이터 셋을 사용합니다. 훈련용 5,000개와 테스트용 1,000개로 이루어져 있으며, 
레이블은 긍정/부정으로 두 가지입니다.   
우선 자연어 데이터를 RNN 모델의 입력으로 사용할 수 있도록 데이터 전 처리를 수행해보겠습니다.    

In [1]:
import json
import numpy as np
import tensorflow as tf
import data_process
from keras.datasets import imdb
from keras.preprocessing import sequence

ModuleNotFoundError: No module named 'data_process'

In [ ]:

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)


In [ ]:

# 학습용 및 평가용 데이터를 불러오고 샘플 문장을 출력합니다.
X_train, y_train, X_test, y_test = data_process.imdb_data_load()

max_review_length = 300



## 영화 리뷰 긍정/부정 분류 RNN 모델 - 모델 학습

일반적으로 RNN 모델은 입력층으로 Embedding 레이어를 먼저 쌓고, RNN 레이어를 몇 개 쌓은 다음, 이후 Dense 레이어를 더 쌓아 완성합니다.

* 임베딩 레이어  
tf.keras.layers.Embedding(input_dim, output_dim, input_length)  

: 들어온 문장을 단어 임베딩(embedding)하는 레이어  
 
input_dim: 들어올 단어의 개수  
 
output_dim: 결과로 나올 임베딩 벡터의 크기(차원)  
 
input_length: 들어오는 단어 벡터의 크기  

* RNN 레이어

tf.keras.layers.SimpleRNN(units)    
: 단순 RNN 레이어  

units: 레이어의 노드 수  

In [ ]:
"""
1. 모델을 구현합니다.
   임베딩 레이어 다음으로 `SimpleRNN`을 사용하여 RNN 레이어를 쌓고 노드의 개수는 5개로 설정합니다. 
   Dense 레이어는 0, 1 분류이기에 노드를 1개로 하고 activation을 'sigmoid'로 설정되어 있습니다.
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(1000, embedding_vector_length, input_length = max_review_length),
    tf.keras.layers.SimpleRNN(5),
    tf.keras.layers.Dense(1, activation='sigmoid') # sigmoid >> 0과1 

# 모델을 확인합니다.
print(model.summary())


In [ ]:
# 학습 방법을 설정합니다.
# loss = 'binary_crossentropy' >> Dense가 0과1의 binary 형태이므로
# metrics >> 분류기 때문에 accuracy
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:


# 학습을 수행합니다.
model_history = model.fit(X_train, y_train, epochs = 3, verbose = 2)

## 영화 리뷰 긍정/부정 분류 RNN 모델 - 평가 및 예측

In [ ]:

"""
1. 평가용 데이터를 활용하여 모델을 평가합니다.
   loss와 accuracy를 계산하고 loss, test_acc에 저장합니다.
"""
loss, test_acc = model.evaluate(X_test, y_test, verbose = 0)

"""
2. 평가용 데이터에 대한 예측 결과를 predictions에 저장합니다.
"""
predictions = model.predict(X_test)

# 모델 평가 및 예측 결과를 출력합니다.
print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
print('예측한 Test Data 클래스 : ',1 if predictions[0]>=0.5 else 0)